In [96]:
import os as os

import tensorflow as tf

import numpy as np

import copy as copy

from sklearn.model_selection import train_test_split

## Data processing

In [60]:
def convertLabel(label): #gives a nb to the aminoacid
    aminoacidList = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS", "ILE",
                    "LEU", "LYS", "MET", "PHE", "PRO", "PYL", "SEL", "SER", "THR", "TRP", "TYR", "VAL"]
    for i in range (0, 22):
        if label == aminoacidList[i]:
            return i
    print("ERROR")

In [61]:
def load_data(filename):


    labelList = []

    with open(filename) as f:

        nbOfSamples = int(f.readline())
        i = 0

        shapes = f.readline() # length of sequence + nb of features
        input_shape = int(shapes.split(",")[1])

        # inputs
        distancesList = np.zeros((nbOfSamples, input_shape))
        previousResiduesList = np.zeros((nbOfSamples, 7))
        
        # outputs
        labelList = np.zeros(nbOfSamples)
        


        while i < nbOfSamples:
            # Extracting labels
            sequence = f.readline().split(",")
            labelList[i] = convertLabel(sequence[8])
            previousResiduesList[i, :] = np.array([convertLabel(sequence[i]) for i in range(0, 7)])

            # Extracting distances
            distancesString = f.readline()
            distancesArray = np.array([float(s) for s in distancesString.split(",")])
            distancesList[i, :] = distancesArray

            # empty line
            f.readline()

            i += 1
    return input_shape, distancesList, previousResiduesList, labelList

## Creating models

### Single input model

In [112]:
def single_input_model():
    # activation function: leaky ReLU
    leakyrelu = lambda x: tf.keras.activations.relu(x, alpha=0.01, max_value=None, threshold=0)

    # creation du reseau de neurones
    model = tf.keras.models.Sequential([

        # hidden layer
        tf.keras.layers.Dense(units = 105, activation = leakyrelu, input_shape = (105,)),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 210, activation = leakyrelu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 210, activation = leakyrelu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 210, activation = leakyrelu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 100, activation = leakyrelu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 75, activation = leakyrelu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 53, activation = leakyrelu),
        tf.keras.layers.Dropout(rate=0.2),

        # final layer
        tf.keras.layers.Dense(units = 22, activation = tf.nn.softmax),
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

    return model

In [113]:
# Define checkppoints
single_input_checkpoint_path = "Single input model/checkpoints/checkpoint.ckpt"
single_input_checkpoint_dir = os.path.dirname(single_input_checkpoint_path)

single_input_cp_callback = tf.keras.callbacks.ModelCheckpoint(
    single_input_checkpoint_path, save_weights_only=True, verbose=1,
    # save weights, every 10-epoch
    period=10)

single_input_latest = tf.train.latest_checkpoint(single_input_checkpoint_dir)

### Double input model

In [114]:
def double_input_model(single_input_model):
    # activation function: leaky ReLU
    leakyrelu = lambda x: tf.keras.activations.relu(x, alpha=0.01, max_value=None, threshold=0)

    # distances branch
    distancesBranch = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units = 105, activation = leakyrelu, input_shape = (105,)),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 210, activation = leakyrelu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 210, activation = leakyrelu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 100, activation = leakyrelu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 75, activation = leakyrelu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 53, activation = leakyrelu),
        tf.keras.layers.Dropout(rate=0.2)
    ])
    
    
    # residues branch   
    residuesBranch = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units = 7, activation = leakyrelu, input_shape = (7,))
    ])
    
    # concatenation
    combinedInput = tf.keras.layers.concatenate([distancesBranch.output, residuesBranch.output])

    x = tf.keras.layers.Dense(units = 30, activation = leakyrelu, input_shape = (59,))(combinedInput)
    x = tf.keras.layers.Dense(units = 22, activation = tf.nn.softmax)(x)
    
    model = tf.keras.models.Model(inputs = [distancesBranch.input, residuesBranch.input], outputs = x)

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

    return model

In [115]:
# Define checkppoints
double_input_checkpoint_path = "Double input model/checkpoints/checkpoint.ckpt"
double_input_checkpoint_dir = os.path.dirname(double_input_checkpoint_path)

double_input_cp_callback = tf.keras.callbacks.ModelCheckpoint(
    double_input_checkpoint_path, save_weights_only=True, verbose=1,
    # save weights, every 50-epoch
    period=50)

double_input_latest = tf.train.latest_checkpoint(double_input_checkpoint_dir)

## Training

### Importing data

In [116]:
# Data processing
input_shape, distances, previousResidues, output = load_data("Data/fullmatrices.csv")


distances_train, distances_test, residues_train, residues_test, y_train, y_test = train_test_split(
    distances, previousResidues, output, test_size=0.2, shuffle = True, random_state=2)

### Training the single input model

#### Creation

In [117]:
single_input_model = single_input_model()

if single_input_latest: single_input_model.load_weights(single_input_latest)

#### Training

In [125]:
single_input_model.fit(distances_train, y_train,
                       validation_data = (distances_test, y_test),
                       shuffle = True,
                       callbacks = [single_input_cp_callback], epochs = 500)

Train on 3264 samples, validate on 817 samples
Epoch 1/500
  32/3264 [..............................] - ETA: 1s - loss: 1.3189 - acc: 0.5625Train on 3264 samples, validate on 817 samples
Epoch 1/500
3264/3264 [==============================] - 1s 270us/sample - loss: 1.6237 - acc: 0.4819 - val_loss: 1.4958 - val_acc: 0.5275
Epoch 2/500
3264/3264 [==============================] - 1s 251us/sample - loss: 1.6347 - acc: 0.4776 - val_loss: 1.5488 - val_acc: 0.5031
Epoch 3/500
3264/3264 [==============================] - 1s 248us/sample - loss: 1.6524 - acc: 0.4694 - val_loss: 1.4770 - val_acc: 0.5300
Epoch 4/500
3264/3264 [==============================] - 1s 265us/sample - loss: 1.7403 - acc: 0.4620 - val_loss: 1.5267 - val_acc: 0.5006
Epoch 5/500
3264/3264 [==============================] - 1s 253us/sample - loss: 1.7540 - acc: 0.4577 - val_loss: 1.6407 - val_acc: 0.4920
Epoch 6/500
3264/3264 [==============================] - 1s 252us/sample - loss: 1.6479 - acc: 0.4810 - val_loss: 1.57


Epoch 00020: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3264/3264 [==============================] - 1s 267us/sample - loss: 1.6310 - acc: 0.4755 - val_loss: 1.4398 - val_acc: 0.5508
Epoch 21/500
3264/3264 [==============================] - 1s 236us/sample - loss: 1.6819 - acc: 0.4703 - val_loss: 1.5181 - val_acc: 0.5422
Epoch 22/500
3264/3264 [==============================] - 1s 238us/sample - loss: 1.6419 - acc: 0.4795 - val_loss: 1.4840 - val_acc: 0.5275
Epoch 23/500
3264/3264 [==============================] - 1s 239us/sample - loss: 1.6593 - acc: 0.4697 - val_loss: 1.5036 - val_acc: 0.5349
Epoch 24/500
3264/3264 [==============================] - 1s 244us/sample - loss: 1.6557 - acc: 0.4782 - val_loss: 1.4739 - val_acc: 0.5471
Epoch 25/500
Epoch 25/500
3264/3264 [==============================] - 1s 246us/sample - loss: 1.6835 - acc: 0.4669 - val_loss: 1.4795 - val_acc: 0.5447
Epoch 26/500
3264/3264 [===

Epoch 40/500
3168/3264 [============================>.] - ETA: 0s - loss: 1.6635 - acc: 0.4782
Epoch 00040: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00040: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3264/3264 [==============================] - 1s 252us/sample - loss: 1.6563 - acc: 0.4804 - val_loss: 1.5232 - val_acc: 0.5312
Epoch 41/500
3264/3264 [==============================] - 1s 238us/sample - loss: 1.6782 - acc: 0.4678 - val_loss: 1.4661 - val_acc: 0.5214
Epoch 42/500
3264/3264 [==============================] - 1s 247us/sample - loss: 1.6865 - acc: 0.4746 - val_loss: 1.4932 - val_acc: 0.5226
Epoch 43/500
3264/3264 [==============================] - 1s 254us/sample - loss: 1.6635 - acc: 0.4730 - val_loss: 1.5368 - val_acc: 0.5080
Epoch 44/500
Epoch 44/500
3264/3264 [==============================] - 1s 246us/sample - loss

Epoch 58/500
3264/3264 [==============================] - 1s 249us/sample - loss: 1.6314 - acc: 0.4804 - val_loss: 1.5157 - val_acc: 0.5288
Epoch 59/500
3264/3264 [==============================] - 1s 246us/sample - loss: 1.7151 - acc: 0.4632 - val_loss: 1.5467 - val_acc: 0.5214
Epoch 60/500
3008/3264 [==========================>...] - ETA: 0s - loss: 1.6920 - acc: 0.4528
Epoch 00060: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00060: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3264/3264 [==============================] - 1s 255us/sample - loss: 1.6940 - acc: 0.4519 - val_loss: 1.6460 - val_acc: 0.4957
Epoch 61/500
3264/3264 [==============================] - 1s 240us/sample - loss: 1.6787 - acc: 0.4617 - val_loss: 1.7911 - val_acc: 0.4676
Epoch 62/500
Epoch 62/500
3264/3264 [==============================] - 1s 249us/sample - loss

Epoch 76/500
Epoch 76/500
3264/3264 [==============================] - 1s 238us/sample - loss: 1.6962 - acc: 0.4580 - val_loss: 1.4750 - val_acc: 0.5422
Epoch 77/500
3264/3264 [==============================] - 1s 246us/sample - loss: 1.6642 - acc: 0.4718 - val_loss: 1.4869 - val_acc: 0.5459
Epoch 78/500
3264/3264 [==============================] - 1s 252us/sample - loss: 1.6968 - acc: 0.4737 - val_loss: 1.5298 - val_acc: 0.5410
Epoch 79/500
3264/3264 [==============================] - 1s 246us/sample - loss: 1.6224 - acc: 0.4862 - val_loss: 1.5627 - val_acc: 0.4957
Epoch 80/500
3200/3264 [============================>.] - ETA: 0s - loss: 1.6657 - acc: 0.4737
Epoch 00080: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00080: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3264/3264 [==============================] - 1s 248us/sample - loss

...

Epoch 423/500
3264/3264 [==============================] - 1s 238us/sample - loss: 1.4690 - acc: 0.5355 - val_loss: 1.3075 - val_acc: 0.5912
Epoch 424/500
3264/3264 [==============================] - 1s 247us/sample - loss: 1.4041 - acc: 0.5426 - val_loss: 1.3273 - val_acc: 0.5985
Epoch 425/500
3264/3264 [==============================] - 1s 241us/sample - loss: 1.4539 - acc: 0.5389 - val_loss: 1.3159 - val_acc: 0.5655
Epoch 426/500
3264/3264 [==============================] - 1s 246us/sample - loss: 1.5436 - acc: 0.5303 - val_loss: 1.3319 - val_acc: 0.5741
Epoch 427/500
3264/3264 [==============================] - 1s 244us/sample - loss: 1.4320 - acc: 0.5463 - val_loss: 1.3137 - val_acc: 0.5900
Epoch 428/500
Epoch 428/500
3264/3264 [==============================] - 1s 246us/sample - loss: 1.6077 - acc: 0.5049 - val_loss: 1.3439 - val_acc: 0.5716
Epoch 429/500
3264/3264 [==============================] - 1s 240us/sample - loss: 1.4817 - acc: 0.5346 - val_loss: 1.2623 - val_acc: 0.5973

3264/3264 [==============================] - 1s 252us/sample - loss: 1.4315 - acc: 0.5423 - val_loss: 1.2411 - val_acc: 0.6144
Epoch 441/500
3264/3264 [==============================] - 1s 242us/sample - loss: 1.4527 - acc: 0.5444 - val_loss: 1.2214 - val_acc: 0.6108
Epoch 442/500
Epoch 442/500
3264/3264 [==============================] - 1s 246us/sample - loss: 1.4720 - acc: 0.5328 - val_loss: 1.2556 - val_acc: 0.6095
Epoch 443/500
3264/3264 [==============================] - 1s 246us/sample - loss: 1.4564 - acc: 0.5297 - val_loss: 1.2789 - val_acc: 0.5961
Epoch 444/500
3264/3264 [==============================] - 1s 248us/sample - loss: 1.5284 - acc: 0.5267 - val_loss: 1.3380 - val_acc: 0.5594
Epoch 445/500
3264/3264 [==============================] - 1s 248us/sample - loss: 1.4678 - acc: 0.5362 - val_loss: 1.2789 - val_acc: 0.6157
Epoch 446/500
3264/3264 [==============================] - 1s 245us/sample - loss: 1.5439 - acc: 0.5193 - val_loss: 1.2814 - val_acc: 0.5949
Epoch 447/500


Epoch 00460: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3264/3264 [==============================] - 1s 252us/sample - loss: 1.4655 - acc: 0.5316 - val_loss: 1.2841 - val_acc: 0.6010
Epoch 461/500
3264/3264 [==============================] - 1s 235us/sample - loss: 1.4317 - acc: 0.5435 - val_loss: 1.2206 - val_acc: 0.6157
Epoch 462/500
3264/3264 [==============================] - 1s 243us/sample - loss: 1.5332 - acc: 0.5233 - val_loss: 1.3841 - val_acc: 0.5606
Epoch 463/500
3264/3264 [==============================] - 1s 242us/sample - loss: 1.5922 - acc: 0.5080 - val_loss: 1.2823 - val_acc: 0.5961
Epoch 464/500
3264/3264 [==============================] - 1s 245us/sample - loss: 1.4986 - acc: 0.5193 - val_loss: 1.3379 - val_acc: 0.5667
Epoch 465/500
3264/3264 [==============================] - 1s 235us/sample - loss: 1.5367 - acc: 0.5294 - val_loss: 1.3402 - val_acc: 0.5716
Epoch 466/500
3264/3264 [==========

3264/3264 [==============================] - 1s 292us/sample - loss: 1.4801 - acc: 0.5349 - val_loss: 1.3145 - val_acc: 0.5900
Epoch 480/500
Epoch 480/500
3040/3264 [==========================>...] - ETA: 0s - loss: 1.4719 - acc: 0.5408
Epoch 00480: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00480: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3264/3264 [==============================] - 1s 264us/sample - loss: 1.4688 - acc: 0.5417 - val_loss: 1.2479 - val_acc: 0.5875
Epoch 481/500
3264/3264 [==============================] - 1s 270us/sample - loss: 1.5030 - acc: 0.5239 - val_loss: 1.3283 - val_acc: 0.5875
Epoch 482/500
3264/3264 [==============================] - 1s 259us/sample - loss: 1.4767 - acc: 0.5389 - val_loss: 1.2821 - val_acc: 0.5875
Epoch 483/500
Epoch 483/500
3264/3264 [==============================] - 1s 264us/sample 

Epoch 498/500
3264/3264 [==============================] - 1s 320us/sample - loss: 1.4722 - acc: 0.5377 - val_loss: 1.3273 - val_acc: 0.5716
Epoch 499/500
3264/3264 [==============================] - 1s 310us/sample - loss: 1.7022 - acc: 0.4850 - val_loss: 1.4772 - val_acc: 0.5349
Epoch 500/500
3200/3264 [============================>.] - ETA: 0s - loss: 1.7164 - acc: 0.4600
Epoch 00500: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00500: saving model to Single input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3264/3264 [==============================] - 1s 282us/sample - loss: 1.7175 - acc: 0.4589 - val_loss: 1.3860 - val_acc: 0.5728


#### Saving

In [119]:
single_input_model.save("Single input model/single_input_model.h5")

### Training the double input model

#### Creation

In [120]:
double_input_model = double_input_model(single_input_model)

if double_input_latest: double_input_model.load_weights(double_input_latest)

#### Training

In [121]:
double_input_model.fit([distances_train, residues_train], y_train,
                       shuffle = True,
                       validation_data = ([distances_test, residues_test], y_test),
                       callbacks = [double_input_cp_callback], epochs = 500)

Train on 3264 samples, validate on 817 samples
Train on 3264 samples, validate on 817 samples
Epoch 1/500
Epoch 1/500
3264/3264 [==============================] - 2s 614us/sample - loss: 0.7058 - acc: 0.8199 - val_loss: 0.7690 - val_acc: 0.8543
Epoch 2/500
3264/3264 [==============================] - 1s 225us/sample - loss: 0.6670 - acc: 0.8260 - val_loss: 0.7362 - val_acc: 0.8825
Epoch 3/500
3264/3264 [==============================] - 1s 224us/sample - loss: 0.6699 - acc: 0.8336 - val_loss: 0.7466 - val_acc: 0.8825
Epoch 4/500
3264/3264 [==============================] - 1s 220us/sample - loss: 0.6978 - acc: 0.8275 - val_loss: 0.7849 - val_acc: 0.8703
Epoch 5/500
3264/3264 [==============================] - 1s 225us/sample - loss: 0.6874 - acc: 0.8336 - val_loss: 0.7227 - val_acc: 0.8862
Epoch 6/500
3264/3264 [==============================] - 1s 226us/sample - loss: 0.6615 - acc: 0.8309 - val_loss: 0.7100 - val_acc: 0.8813
Epoch 7/500
3264/3264 [==============================] - 1s 

3264/3264 [==============================] - 1s 258us/sample - loss: 0.6521 - acc: 0.8355 - val_loss: 0.7202 - val_acc: 0.8788
Epoch 25/500
3264/3264 [==============================] - 1s 256us/sample - loss: 0.6639 - acc: 0.8281 - val_loss: 0.6936 - val_acc: 0.8788
Epoch 26/500
Epoch 26/500
3264/3264 [==============================] - 1s 251us/sample - loss: 0.6413 - acc: 0.8379 - val_loss: 0.7553 - val_acc: 0.8629
Epoch 27/500
3264/3264 [==============================] - 1s 226us/sample - loss: 0.6190 - acc: 0.8471 - val_loss: 0.6821 - val_acc: 0.8862
Epoch 28/500
3264/3264 [==============================] - 1s 221us/sample - loss: 0.6148 - acc: 0.8526 - val_loss: 0.6696 - val_acc: 0.8984
Epoch 29/500
3264/3264 [==============================] - 1s 228us/sample - loss: 0.6796 - acc: 0.8263 - val_loss: 0.7118 - val_acc: 0.8727
Epoch 30/500
3264/3264 [==============================] - 1s 228us/sample - loss: 0.7340 - acc: 0.8125 - val_loss: 0.7632 - val_acc: 0.8629
Epoch 31/500
3264/32

3264/3264 [==============================] - 1s 265us/sample - loss: 0.6669 - acc: 0.8364 - val_loss: 0.7089 - val_acc: 0.8898
Epoch 48/500
3264/3264 [==============================] - 1s 233us/sample - loss: 0.6662 - acc: 0.8281 - val_loss: 0.7350 - val_acc: 0.8764
Epoch 49/500
3264/3264 [==============================] - 1s 243us/sample - loss: 0.6962 - acc: 0.8251 - val_loss: 0.7364 - val_acc: 0.8727
Epoch 50/500
3136/3264 [===========================>..] - ETA: 0s - loss: 0.6678 - acc: 0.8310
Epoch 00050: saving model to Double input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00050: saving model to Double input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3264/3264 [==============================] - 2s 611us/sample - loss: 0.6685 - acc: 0.8303 - val_loss: 0.6948 - val_acc: 0.8727
Epoch 51/500
3264/3264 [==============================] - 1s 244us/sample - loss: 0.7023 - acc: 0.8211 - v

3264/3264 [==============================] - 1s 248us/sample - loss: 0.6404 - acc: 0.8453 - val_loss: 0.6875 - val_acc: 0.8972
Epoch 69/500
3264/3264 [==============================] - 1s 251us/sample - loss: 0.6235 - acc: 0.8465 - val_loss: 0.6813 - val_acc: 0.8886
Epoch 70/500
3264/3264 [==============================] - 1s 258us/sample - loss: 0.6305 - acc: 0.8425 - val_loss: 0.7297 - val_acc: 0.8629
Epoch 71/500
3264/3264 [==============================] - 1s 282us/sample - loss: 0.6878 - acc: 0.8290 - val_loss: 0.6617 - val_acc: 0.8886
Epoch 72/500
3264/3264 [==============================] - 1s 256us/sample - loss: 0.6157 - acc: 0.8431 - val_loss: 0.6617 - val_acc: 0.8874
Epoch 73/500
3264/3264 [==============================] - 1s 299us/sample - loss: 0.6391 - acc: 0.8373 - val_loss: 0.8021 - val_acc: 0.8286
Epoch 74/500
3264/3264 [==============================] - 1s 262us/sample - loss: 0.6825 - acc: 0.8254 - val_loss: 0.9965 - val_acc: 0.8201
Epoch 75/500
3264/3264 [=========

Epoch 92/500
3264/3264 [==============================] - 1s 220us/sample - loss: 0.7302 - acc: 0.8226 - val_loss: 0.7043 - val_acc: 0.8935
Epoch 93/500
Epoch 93/500
3264/3264 [==============================] - 1s 219us/sample - loss: 0.6436 - acc: 0.8382 - val_loss: 0.7204 - val_acc: 0.8641
Epoch 94/500
Epoch 94/500
3264/3264 [==============================] - 1s 218us/sample - loss: 0.7200 - acc: 0.8205 - val_loss: 0.6895 - val_acc: 0.8996
Epoch 95/500
Epoch 95/500
3264/3264 [==============================] - 1s 215us/sample - loss: 0.6227 - acc: 0.8395 - val_loss: 0.7315 - val_acc: 0.8690
Epoch 96/500
3264/3264 [==============================] - 1s 222us/sample - loss: 0.7359 - acc: 0.8153 - val_loss: 0.7163 - val_acc: 0.8788
Epoch 97/500
3264/3264 [==============================] - 1s 227us/sample - loss: 0.7205 - acc: 0.8254 - val_loss: 0.7599 - val_acc: 0.8507
Epoch 98/500
3264/3264 [==============================] - 1s 223us/sample - loss: 0.7104 - acc: 0.8061 - val_loss: 0.8724

...


Epoch 00400: saving model to Double input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3264/3264 [==============================] - 1s 336us/sample - loss: 0.5642 - acc: 0.8597 - val_loss: 0.5729 - val_acc: 0.9082
Epoch 401/500
3264/3264 [==============================] - 1s 251us/sample - loss: 0.5717 - acc: 0.8575 - val_loss: 0.5888 - val_acc: 0.8984
Epoch 402/500
3264/3264 [==============================] - 1s 252us/sample - loss: 0.5355 - acc: 0.8670 - val_loss: 0.6563 - val_acc: 0.8935
Epoch 403/500
3264/3264 [==============================] - 1s 257us/sample - loss: 0.5421 - acc: 0.8597 - val_loss: 0.6430 - val_acc: 0.9045
Epoch 404/500
Epoch 404/500
3264/3264 [==============================] - 1s 258us/sample - loss: 0.5455 - acc: 0.8695 - val_loss: 0.5895 - val_acc: 0.9021
Epoch 405/500
3264/3264 [==============================] - 1s 252us/sample - loss: 0.5239 - acc: 0.8713 - val_loss: 0.6006 - val_acc: 0.9094
Epoch 406/500
3264/32

3264/3264 [==============================] - 1s 243us/sample - loss: 0.5452 - acc: 0.8621 - val_loss: 0.5676 - val_acc: 0.9180
Epoch 423/500
3264/3264 [==============================] - 1s 223us/sample - loss: 0.5120 - acc: 0.8759 - val_loss: 0.5790 - val_acc: 0.9094
Epoch 424/500
3264/3264 [==============================] - 1s 244us/sample - loss: 0.5943 - acc: 0.8554 - val_loss: 0.6163 - val_acc: 0.8947
Epoch 425/500
Epoch 425/500
3264/3264 [==============================] - 1s 259us/sample - loss: 0.6011 - acc: 0.8569 - val_loss: 0.5718 - val_acc: 0.9009
Epoch 426/500
Epoch 426/500
3264/3264 [==============================] - 1s 261us/sample - loss: 0.6431 - acc: 0.8416 - val_loss: 0.6002 - val_acc: 0.8947
Epoch 427/500
3264/3264 [==============================] - 1s 258us/sample - loss: 0.6293 - acc: 0.8539 - val_loss: 0.6086 - val_acc: 0.9106
Epoch 428/500
Epoch 428/500
3264/3264 [==============================] - 1s 266us/sample - loss: 0.5639 - acc: 0.8658 - val_loss: 0.5701 - v

Epoch 446/500
3264/3264 [==============================] - 1s 223us/sample - loss: 0.5462 - acc: 0.8732 - val_loss: 0.6000 - val_acc: 0.9070
Epoch 447/500
3264/3264 [==============================] - 1s 226us/sample - loss: 0.5324 - acc: 0.8676 - val_loss: 0.6066 - val_acc: 0.9094
Epoch 448/500
3264/3264 [==============================] - 1s 225us/sample - loss: 0.4887 - acc: 0.8716 - val_loss: 0.5259 - val_acc: 0.9155
Epoch 449/500
3264/3264 [==============================] - 1s 224us/sample - loss: 0.4976 - acc: 0.8808 - val_loss: 0.5477 - val_acc: 0.9253
Epoch 450/500
Epoch 450/500
3104/3264 [===========================>..] - ETA: 0s - loss: 0.5192 - acc: 0.8721
Epoch 00450: saving model to Double input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00450: saving model to Double input model/checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3264/3264 [==============================] - 1s 327us/sample 

3264/3264 [==============================] - 1s 225us/sample - loss: 0.5618 - acc: 0.8618 - val_loss: 0.7746 - val_acc: 0.8433
Epoch 467/500
3264/3264 [==============================] - 1s 226us/sample - loss: 0.6243 - acc: 0.8413 - val_loss: 0.6485 - val_acc: 0.9009
Epoch 468/500
3264/3264 [==============================] - 1s 223us/sample - loss: 0.5814 - acc: 0.8563 - val_loss: 0.6142 - val_acc: 0.8947
Epoch 469/500
Epoch 469/500
3264/3264 [==============================] - 1s 223us/sample - loss: 0.5726 - acc: 0.8569 - val_loss: 0.6216 - val_acc: 0.8984
Epoch 470/500
Epoch 470/500
3264/3264 [==============================] - 1s 224us/sample - loss: 0.5481 - acc: 0.8686 - val_loss: 0.5615 - val_acc: 0.9106
Epoch 471/500
3264/3264 [==============================] - 1s 228us/sample - loss: 0.5624 - acc: 0.8609 - val_loss: 0.6207 - val_acc: 0.9009
Epoch 472/500
3264/3264 [==============================] - 1s 227us/sample - loss: 0.5780 - acc: 0.8624 - val_loss: 0.5823 - val_acc: 0.9106

3264/3264 [==============================] - 1s 227us/sample - loss: 0.5969 - acc: 0.8609 - val_loss: 0.6188 - val_acc: 0.8960
Epoch 490/500
3264/3264 [==============================] - 1s 225us/sample - loss: 0.5388 - acc: 0.8646 - val_loss: 0.5776 - val_acc: 0.9106
Epoch 491/500
3264/3264 [==============================] - 1s 226us/sample - loss: 0.5680 - acc: 0.8655 - val_loss: 0.6282 - val_acc: 0.8923
Epoch 492/500
3264/3264 [==============================] - 1s 243us/sample - loss: 0.8327 - acc: 0.8131 - val_loss: 0.6534 - val_acc: 0.8996
Epoch 493/500
Epoch 493/500
3264/3264 [==============================] - 1s 232us/sample - loss: 0.5419 - acc: 0.8710 - val_loss: 0.5964 - val_acc: 0.9155
Epoch 494/500
3264/3264 [==============================] - 1s 228us/sample - loss: 0.5500 - acc: 0.8615 - val_loss: 0.5732 - val_acc: 0.9119
Epoch 495/500
3264/3264 [==============================] - 1s 230us/sample - loss: 0.5266 - acc: 0.8676 - val_loss: 0.5846 - val_acc: 0.9082
Epoch 496/500

#### Saving

In [123]:
double_input_model.save("Double input model/double_input_model.h5")